In [1]:
from __future__ import division

In [2]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp
import urllib
import os

In [3]:
from sqlalchemy import create_engine

In [4]:
pd.set_option('display.max_columns', None)
Gb = 1000000000

In [5]:
temp_csv = 'c:'+os.sep+'temp'+os.sep+'temp.log'

In [6]:
log_file = 'c:'+os.sep+'temp'+os.sep+'hk-ssg140.log-20150922'

In [7]:
disk_engine = create_engine('sqlite:///c:\\temp\\access_log.db')

In [8]:
#store = pd.HDFStore('access_log.h5')

In [9]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [10]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [11]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [12]:
#squid = sq.read_squid_log('access.log')

In [13]:
#squid.tail()

In [14]:
#safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [15]:
#access = squid[squid['URL'].str.contains(safe_websites) == False]

In [16]:
#sum(squid.bytes)/Gb

In [17]:
def clean_juniper_file(infile, outfile):
    with open(infile, 'r') as in_file:
        with open(outfile, 'w') as out_file:
            for line in in_file:
                if ('system-notification-00257(traffic)' in line):                                                
                    idx_1 = line.find('service=')
                    idx_2 = line.find(' proto=')
                    if (idx_1 and idx_2):
                        out_file.write(line[:idx_1]+line[idx_1:idx_2].replace(' ', '')+line[idx_2:])

In [18]:
clean_juniper_file(log_file, temp_csv)

In [19]:
def open_csv(filename, chunksize=20000):
    
    df = pd.DataFrame()
    
    for c in pd.read_table(filename, sep='\s{1,}|"', chunksize=chunksize, iterator=True, engine='python', index_col=False, header=None, parse_dates=[[0, 1, 2], [9,10]]):        
        df = pd.concat([df, c])        
        
    print(str(len(df)) + " rows processed.")
    
    return df

def juniper_log_cleansing(x):
    if str(x).find('=') > 0:
        return str(x)[str(x).find('=')+1:]
    elif str(x).find(':') > 0:
        return str(x)[:str(x).find(':')]    
    else:
        return x

def read_syslog_juniper(filename):
       
    df = open_csv(filename)
    
    df = df[df[7].str.contains('information') == False]
    
    # restructure the dataframe
    i = [0]
    i.extend(range(3, 4))
    i.extend([1])
    i.extend(range(9, 13))
    i.extend([14])
    i.extend([16])
    i.extend(range(17, 25))
    df = df.iloc[:, i]
    
    df.columns = ['time', 'device_name', 'traffic_start', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port', 'remarks']
    df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']] = df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']].applymap(juniper_log_cleansing)
    df[['policy_no', 'protocol', 'sent_size', 'received_size', 'src_port', 'dst_port']] = df[['policy_no', 'protocol', 'sent_size', 'received_size', 'src_port', 'dst_port']].convert_objects(convert_numeric = True)
    
    df = df.reset_index(drop=True)
    
    return df

def summary_groups(grouped):
    for name, group in grouped:
        print group
        

In [20]:
juniper = read_syslog_juniper(temp_csv)

1549682 rows processed.


In [21]:
juniper.head()

,time,device_name,traffic_start,duration,policy_no,service,protocol,src_zone,dst_zone,action,sent_size,received_size,src_address,dst_address,src_port,dst_port,remarks
0,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,216.69.185.8,57524,53,src-xlated
1,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,192.33.14.30,57436,53,src-xlated
2,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,37,snmp,17,HK-DMZ,HK-LAN,Deny,0,50,172.21.200.230,172.20.78.123,50835,161,session_id=0
3,2015-09-17 04:02:03,hk-SSG140,2015-09-17 04:13:56,3,38,https,6,HK-LAN,HK-DMZ,Permit,1271,2001,172.20.78.77,172.20.113.3,58776,443,src-xlated
4,2015-09-17 04:02:03,hk-SSG140,2015-09-17 04:13:56,3,470,tcp/port:47337,6,HK-LAN,HK-DMZ,Permit,95,58,172.20.78.115,172.24.66.31,443,47337,src-xlated


0     17092015
1     17092015
2     17092015
3     17092015
4     17092015
5     17092015
6     17092015
7     17092015
8     17092015
9     17092015
10    17092015
11    17092015
12    17092015
13    17092015
14    17092015
...
1549667    18092015
1549668    18092015
1549669    18092015
1549670    18092015
1549671    18092015
1549672    18092015
1549673    18092015
1549674    18092015
1549675    18092015
1549676    18092015
1549677    18092015
1549678    18092015
1549679    18092015
1549680    18092015
1549681    18092015
Name: time, Length: 1549682, dtype: object

In [41]:
juniper['date'] = juniper['time'].apply(lambda x: x.strftime('%Y-%m-%d'))
juniper['total_size'] = juniper['sent_size'] + juniper['received_size']

In [42]:
juniper.head()

,time,device_name,traffic_start,duration,policy_no,service,protocol,src_zone,dst_zone,action,sent_size,received_size,src_address,dst_address,src_port,dst_port,remarks,date,total_size
0,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,216.69.185.8,57524,53,src-xlated,2015-09-17,0
1,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,192.33.14.30,57436,53,src-xlated,2015-09-17,0
2,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,37,snmp,17,HK-DMZ,HK-LAN,Deny,0,50,172.21.200.230,172.20.78.123,50835,161,session_id=0,2015-09-17,50
3,2015-09-17 04:02:03,hk-SSG140,2015-09-17 04:13:56,3,38,https,6,HK-LAN,HK-DMZ,Permit,1271,2001,172.20.78.77,172.20.113.3,58776,443,src-xlated,2015-09-17,3272
4,2015-09-17 04:02:03,hk-SSG140,2015-09-17 04:13:56,3,470,tcp/port:47337,6,HK-LAN,HK-DMZ,Permit,95,58,172.20.78.115,172.24.66.31,443,47337,src-xlated,2015-09-17,153


In [43]:
grouped = juniper.groupby(['date', 'src_address'])

In [44]:
grouped.ngroups

1275

In [45]:
total = 0
for i in grouped.groups.keys():
    subtotal = np.nansum(juniper.iloc[grouped.groups[i]].sent_size)
    #print (i + ': ' + str(subtotal))
    total += subtotal
    #print i
    #print grouped.groups[i]
print 'total: ' + str(total/Gb) + ' Gb'

total: 21.301632089 Gb


In [46]:
temp_result = []
for d, i in grouped.groups.keys():
    temp_result.append([d, i, len(juniper.iloc[grouped.groups[(d, i)]]), np.sum(juniper.iloc[grouped.groups[(d, i)]]).total_size])

In [47]:
sqldb = pd.DataFrame(temp_result, columns=['date', 'ip', 'count', 'total_size'])

In [48]:
sqldb.head()

,date,ip,count,total_size
0,2015-09-17,111.248.115.140,1,20
1,2015-09-17,172.20.114.182,9,1323484
2,2015-09-17,119.56.124.59,2,163451
3,2015-09-17,117.136.40.208,99,287993
4,2015-09-17,59.38.97.78,58,1506649


In [49]:
sqldb.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...], dtype='int64')

In [50]:
sqldb.to_sql('data', disk_engine, index=False, if_exists='append')

In [ ]:
#for k in grouped.groups.keys():
#    print k

In [ ]:
#len(juniper.iloc[483167:483169, :])

In [ ]:
#juniper.iloc[483167:483169, :]

In [ ]:
#juniper.iloc[925410,]

In [ ]:
#pd.DataFrame.sum(juniper[['sent_size']])

In [ ]:
#juniper